In [6]:
import yaml
import os
import pathlib
import json

In [7]:
refdict = yaml.load(open('BH76_ref_energies.yaml','r'), Loader=yaml.Loader)
kcal = 627.509
all_mols = {}

In [8]:
def process_files(input_dir):
    mad_energies = {}
    # Iterate through the directory and its subdirectories
    for root, dirs, files in os.walk(input_dir):
        for dir in dirs:
            mol_name = dir
            all_mols[mol_name] = True

        for file in files:
            if file == "struc.calc_info.json": 
                
                input_file_path = os.path.join(root, file)
                mol_name = input_file_path.split('/')[5]

                f = open(input_file_path)
                data = json.load(f)
                #print(mol_name, data['return_energy'])
                mad_energies[mol_name] = data['return_energy']
    
    return mad_energies

In [9]:
path = os.path.abspath(os.getcwd())
input_directory = path+'/pbe'  # Change this to your desired input directory
mad_pbehf = process_files(input_directory)

In [10]:
print(len(mad_pbehf.keys()))

72


In [19]:
print('MISSING: ')
cmdict = yaml.load(open('BH76_chg_2s.yaml','r'), Loader=yaml.Loader)
for i in all_mols.keys():
    if i in mad_pbehf.keys():
        pass
    else:
        #print(i, "\t", cmdict[i])
        print(i)

MISSING: 
fch3clcomp2
hoch3fts
fch3clts
f-
hoch3fcomp1
hoch3fcomp2
oh-
clch3clcomp
fch3fts
cl-
clch3clts
h
fch3fcomp
fch3clcomp1
.ipynb_checkpoints


In [ ]:
print(refdict[1])

In [ ]:
# only for debug use
h_pbe = -0.4999410227691473
h_hf  = -0.4999535118271135

In [ ]:
# iterate over Refdict, stoich keys lookup in madness energies dict, calc bh according to stoich, scale to kcal
#
# !! code does not support restart from h single electron wf, pbe@hf for h does not work
# here, TEMPORARILY, use hf/pbe energy instead
# error more fundamental - f- oh- also missing - CHARGE needs to be accounted for
pbe_bh76_mad = {}
mad_pbehf['h'] = h_hf
for bh in refdict.keys():
    e_bh = 0   
    for system in refdict[bh]['Stoich'].keys():
        stoich_fac = refdict[bh]['Stoich'][system]
        e_bh += stoich_fac * mad_pbehf[system]
        #print(e_bh)

    print()
    pbe_bh76_mad[bh] = e_bh * kcal
    print(bh, pbe_bh76_mad[bh])

In [ ]:
e_bh = 0   
for system in refdict[37]['Stoich'].keys():
    stoich_fac = refdict[37]['Stoich'][system]
    e_bh += stoich_fac * mad_pbehf[system]
    #print(e_bh)

print()
pbe_bh76_mad[37] = e_bh * kcal
print(37, pbe_bh76_mad[37], 48.10 - pbe_bh76_mad[37])

In [ ]:
# 37 is reproducible, need to figure out systems that went wrong and calc again
# maybe also run pbe only calc for ref (background, everything here now

In [ ]:
print(pbe_bh76_mad)

In [ ]:
for i in pbe_bh76_mad.keys():
    print(i, pbe_bh76_mad[i] - refdict[i]['Ref'])

In [ ]:
# this is only slightly different (.2 ish) from LCAO - good?